**reference** <br>
 : https://github.com/niconielsen32/ComputerVision/tree/master/StereoVisionDepthEstimation 

import module

In [1]:
import sys
import cv2
import numpy as np
import time
import imutils
from matplotlib import pyplot as plt

# Function for stereo vision and depth estimation
import triangulation as tri
import calibration

### stereo vision set-up parameters

In [2]:
frame_rate = 23 # 120     # Camera frame rate (maximum at 120 fps)
B = 24               # Distance between the cameras [cm]
f = 10               # Camera lense's focal length [mm]
alpha = 82.1 # 70.42 # 82.1        # Camera field of view in the horisontal plane [degrees]

### depth estimation TEST with fixed-point

* 점 좌표 설정

In [ ]:
# right_point = [190,330] # 노란점 1 
# left_point = [237,337]
# right_point = [251,271] # 노란점 2 
# left_point = [295,276]
right_point = [339,369] # 노란점 3 
left_point = [389,379]
# right_point = [365,304] # 노란점 4 
# left_point = [415,315]

# right_point = [232,300] # 검은 점 1 
# left_point = [289,311]
# right_point = [56,157] # steroL0.png - 체커보드 0,0 점 
# left_point = [318,166]

In [16]:
right_point = [283,429] # 탁구대 중앙 아래 점 
left_point = [363,433]

* 점의 depth 추출

In [18]:
# Open both cameras
cap_right = cv2.VideoCapture(2, cv2.CAP_DSHOW)                    
cap_left =  cv2.VideoCapture(1, cv2.CAP_DSHOW)

# Main loop for depth estimation using stereo vision
while(cap_right.isOpened() and cap_left.isOpened()):
    start = time.time()
    
    succes_right, frame_right = cap_right.read()
    succes_left, frame_left = cap_left.read()

    ################## CALIBRATION #########################################################

#     c

    ########################################################################################
 
    # If cannot catch any frame, break
    if not succes_right or not succes_left:                    
        break

#     start = time.time()
        
    ################## CALCULATING DEPTH #########################################################
    
    # Function to calculate depth of object. Outputs vector of all depths in case of several balls.
    # All formulas used to find depth is in video presentaion
    depth = tri.find_depth(right_point, left_point, frame_right, frame_left, B, f, alpha) 
    
    cv2.putText(frame_right, "Distance: " + str(round(depth,1)), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0),3)
    cv2.putText(frame_left, "Distance: " + str(round(depth,1)), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0),3)
    # Multiply computer value with 205.8 to get real-life depth in [cm]. The factor was found manually.
    print("Depth: ", str(round(depth,1)))
    
    ########################################################################################

    end = time.time()
    totalTime = end - start
    if totalTime != 0:
        fps = 1 / totalTime
        #print("FPS: ", fps)
        cv2.putText(frame_right, f'FPS: {int(fps)}', (20,450), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 2)
        cv2.putText(frame_left, f'FPS: {int(fps)}', (20,450), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 2)  
    else:
        print("totalTime converges to zero")     
    
    # Show the frames
    cv2.imshow("frame right", frame_right) 
    cv2.imshow("frame left", frame_left)
    
    # Hit "q" to close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release and destroy all windows before termination
cap_right.release()
cap_left.release()

cv2.destroyAllWindows()

### depth map SGBM TEST (fixed-point)

In [ ]:
# Open both cameras
cap_right = cv2.VideoCapture(2, cv2.CAP_DSHOW)                    
cap_left =  cv2.VideoCapture(1, cv2.CAP_DSHOW)

# Main loop for depth estimation using stereo vision
while(cap_right.isOpened() and cap_left.isOpened()):
    start = time.time()
    
    succes_right, frame_right = cap_right.read()
    succes_left, frame_left = cap_left.read()

    # 이미지 크기와 타입 일치시키기
    height, width = frame_left.shape[:2]
    frame_right = cv2.resize(frame_right, (width, height))

    # 스테레오 비전을 위한 SGBM(Semi-Global Block Matching) 알고리즘 생성
    stereo = cv2.StereoSGBM_create(
        minDisparity=0,
        numDisparities=16,  # 최대 차이값 범위 (16의 배수로 설정)
        blockSize=5,
        P1=8 * 3 * 5 ** 2,
        P2=32 * 3 * 5 ** 2,
        disp12MaxDiff=1,
        uniquenessRatio=10,
        speckleWindowSize=100,
        speckleRange=32,
        preFilterCap=63
    )

    # 스테레오 이미지를 흑백으로 변환
    left_gray = cv2.cvtColor(frame_left, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(frame_right, cv2.COLOR_BGR2GRAY)

    # 스테레오 매칭 수행
    disparity = stereo.compute(left_gray, right_gray)

    # 깊이 맵을 0-255 사이로 정규화
    normalized_disparity = cv2.normalize(disparity, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # Show the frames
    cv2.imshow("frame right", frame_right) 
    cv2.imshow("frame left", frame_left)
    
    # Hit "q" to close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release and destroy all windows before termination
cap_right.release()
cap_left.release()

cv2.destroyAllWindows()